In [2]:
import numpy as np
import imutils
import cv2
import math

# load cell image
image = cv2.imread('pm100%.jpg')

# load the image and convert it to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Gaussian blurring to remove noise use 5x5 kernel
#blurred = cv2.GaussianBlur(gray, (5, 5), 0)
blurred=cv2.GaussianBlur(gray, (5, 5), 0)
# show the original and blurred images
cv2.imshow("Original", image)
cv2.imshow("Blurred", blurred)

# compute a "wide", "mid-range", and "tight" threshold for the edges (Edge Detection)
wide = cv2.Canny(blurred, 10, 200)
mid = cv2.Canny(blurred, 30, 150)
tight = cv2.Canny(blurred, 240, 250)

# show the edge maps - from images, mid threshold looks to be the best choice
cv2.imshow("Wide Edge Map", wide)
# cv2.imshow("Mid Edge Map", mid)
# cv2.imshow("Tight Edge Map", tight)


# dilation - do it on mid to close contour gaps (Morphological Transformation)
dilated = cv2.dilate(mid.copy(), None, iterations=1)
cv2.imshow("Dilated {} times".format(1), dilated)

# find all contours in the image and draw ALL contours on the image
cnts = cv2.findContours(dilated.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
cnts = imutils.grab_contours(cnts)
clone = image.copy()
cv2.drawContours(clone, cnts, -1, (0, 255, 0), 1)
print("Found {} contours".format(len(cnts)))

# show the output image
cv2.imshow("All Contours", clone)

# calculate confluence area
total_area = 0
for i, c in enumerate(cnts):
    area = cv2.contourArea(c)
    total_area += area

total_image_area = mid.shape[0] * mid.shape[1]
print(mid.shape)
confluence = (total_area / total_image_area) * 100
if confluence > 100:
    confluence = 100
print(f"Confluence: {confluence}")
cv2.waitKey(0)

Found 389 contours
(173, 234)
Area of total image: 40482
Area enclosed by contours: 44723.5
Confluence: 100
proliferation in hours: 20
Your cells will be overgrown in 0 Days and 0 Hours
Primary Myoblast Cell


In [79]:
#Estimating the confluency overgrown   
Day=[]
Hours=[]
proliferation_rate=float(input('proliferation in hours: '))
day_conversion=proliferation_rate/24
overgrown=100.1

for n in range(100):
    while confluence*2**n<overgrown:
        Day=math.floor(day_conversion* 2**(n-1))
        Hours=math.floor(((overgrown-(confluence*2**n))*proliferation_rate)/(confluence*2**n))
        break

if Hours>=24 and Hours<=48:
    Day=Day + 1
    Hours=Hours-24

if Hours>=48 and Hours<=72:
    Day=Day + 2
    Hours=Hours-48
    
print(f'Your cells will be overgrown in {round(Day,1)} Days and {round(Hours,2)} Hours')

proliferation in hours: 24
Your cells will be overgrown in 0 Days and 0 Hours


In [76]:
#predicting which cell type it is

cnts = cv2.findContours(dilated.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
cnts = imutils.grab_contours(cnts)
total_area = 0
hull_total=0
for i, c in enumerate(cnts):
    area = cv2.contourArea(c)
    hull=cv2.convexHull(c)
    hull_area = cv2.contourArea(hull)
    total_area += area
    hull_total += hull_area

solidity = total_area/hull_total
round(solidity,2)

if solidity>0.79:
    print('Primary Myoblast Cell')
else:
    print('NIH 3T3 Cell')

NIH 3T3 Cell
